# Actor Critic for Cartpole

In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.distributions import Categorical

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import gym

### Cartpole environment

In [3]:
env = gym.make('CartPole-v1')

### Vanilla Actor Critic agent

In [ ]:
class ActorCritic():
    
    def __init__(self, env, actor_lr=0.01, critic_lr=0.01, dr=0.95):
        # Cartpole environment
        self.cartpole_env = env
        
        # Learning rate
        self.lr = lr
        # Discount rate
        self.dr = dr
        
        # Actor model: learns policy function
        #   - Inputs: 4 real values: [Cart Position, Cart Velocity, Pole Angle, Pole Angular Velocity]
        #   - 3 fully connected layers
        #   - final softmax activation to get probability distribution
        #   - Outputs: Probability of taking the two actions: [Push cart to left, Push cart to right]
        self.actor = nn.Sequential(
            nn.Linear(in_features=4, out_features=10),
            nn.ReLU(),
            nn.Linear(in_features=10, out_features=2),
            nn.ReLU(),
            nn.Linear(in_features=2, out_features=2),
            nn.Softmax()
        )
        self.actor_optim = torch.optim.Adam(self.policy.parameters(), lr=self.actor_lr)
        
        # Critic model: learning value function
        #   - Inputs: 4 real values: [Cart Position, Cart Velocity, Pole Angle, Pole Angular Velocity]
        #   - 3 fully connected layers
        #   - Outputs: Value of each action in current state
        self.critic = nn.Sequential(
            nn.Linear(in_features=4, out_features=10),
            nn.ReLU(),
            nn.Linear(in_features=10, out_features=2),
            nn.ReLU(),
            nn.Linear(in_features=2, out_features=2)
        )
        self.critic_optim = torch.optim.Adam(self.)